#Imports

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


#Data Preperation

download data

In [ ]:
! kaggle datasets download hugomathien/soccer

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


unzip data

In [ ]:
!unzip soccer.zip -d data

Archive:  soccer.zip
replace data/database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Read all tables to pandas dataframes

In [ ]:
cnx = sqlite3.connect('data/database.sqlite')

Country = pd.read_sql_query("SELECT * FROM Country", cnx)
League = pd.read_sql_query("SELECT * FROM League", cnx)
Match = pd.read_sql_query("SELECT * FROM Match", cnx)
Player = pd.read_sql_query("SELECT * FROM Player", cnx)
Player_Attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
Team = pd.read_sql_query("SELECT * FROM Team", cnx)
Team_Attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", cnx)

Clean match data and prepare it

In [ ]:
col_list = ['season', 'date', 'home_team_api_id', 'away_team_api_id',
            'home_team_goal', 'away_team_goal', 'home_player_1' , 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7' ,
             'home_player_8' , 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
             'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8','away_player_9', 'away_player_10', 'away_player_11']
player_col_list = ['player_api_id', 'overall_rating']
#68.6 mean
matches = Match[col_list]
player_ratings = Player_Attributes[player_col_list]
for row in range(len(matches)):
  for col in range(6,28):
    if(matches.iat[row,col] > 0):
      if matches.iat[row,col] in player_ratings['player_api_id']:
        matches.iat[row,col] = player_ratings['overall_rating'].loc[matches.iat[row,col]]
      else:
        matches.iat[row,col] = 68.6
    else:
      matches.iat[row,col] = 68.6

print(matches)

          season                 date  home_team_api_id  away_team_api_id  \
0      2008/2009  2008-08-17 00:00:00              9987              9993   
1      2008/2009  2008-08-16 00:00:00             10000              9994   
2      2008/2009  2008-08-16 00:00:00              9984              8635   
3      2008/2009  2008-08-17 00:00:00              9991              9998   
4      2008/2009  2008-08-16 00:00:00              7947              9985   
...          ...                  ...               ...               ...   
25974  2015/2016  2015-09-22 00:00:00             10190             10191   
25975  2015/2016  2015-09-23 00:00:00              9824             10199   
25976  2015/2016  2015-09-23 00:00:00              9956             10179   
25977  2015/2016  2015-09-22 00:00:00              7896             10243   
25978  2015/2016  2015-09-23 00:00:00             10192              9931   

       home_team_goal  away_team_goal  home_player_1  home_player_2  \
0   

create team averages

In [ ]:
home = ['home_player_1' , 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7' ,
             'home_player_8' , 'home_player_9', 'home_player_10', 'home_player_11']
away = [ 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
             'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8','away_player_9', 'away_player_10', 'away_player_11']
matches['home_avg'] = matches[home].mean(axis=1)
matches['away_avg'] = matches[away].mean(axis=1)
for k in home:
  matches = matches.drop(k, axis=1)
for k in away:
  matches = matches.drop(k, axis=1)
print(matches)

          season                 date  home_team_api_id  away_team_api_id  \
0      2008/2009  2008-08-17 00:00:00              9987              9993   
1      2008/2009  2008-08-16 00:00:00             10000              9994   
2      2008/2009  2008-08-16 00:00:00              9984              8635   
3      2008/2009  2008-08-17 00:00:00              9991              9998   
4      2008/2009  2008-08-16 00:00:00              7947              9985   
...          ...                  ...               ...               ...   
25974  2015/2016  2015-09-22 00:00:00             10190             10191   
25975  2015/2016  2015-09-23 00:00:00              9824             10199   
25976  2015/2016  2015-09-23 00:00:00              9956             10179   
25977  2015/2016  2015-09-22 00:00:00              7896             10243   
25978  2015/2016  2015-09-23 00:00:00             10192              9931   

       home_team_goal  away_team_goal   home_avg   away_avg  
0            

<ipython-input-7-ee8aa0f7a4fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['home_avg'] = matches[home].mean(axis=1)
<ipython-input-7-ee8aa0f7a4fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['away_avg'] = matches[away].mean(axis=1)


Add team attributes to matches df

In [ ]:
cols = ['team_api_id', 'date', 'buildUpPlaySpeed',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'chanceCreationPassing',
       'chanceCreationCrossing',
       'chanceCreationShooting',
       'defencePressure',
       'defenceAggression',
       'defenceTeamWidth']

team_att = Team_Attributes[cols]
matches[['h_buildUpPlaySpeed',
       'h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_chanceCreationPassing',
       'h_chanceCreationCrossing',
       'h_chanceCreationShooting',
       'h_defencePressure',
       'h_defenceAggression',
       'h_defenceTeamWidth','a_buildUpPlaySpeed',
       'a_buildUpPlayDribblingClass', 'a_buildUpPlayPassing',
       'a_chanceCreationPassing',
       'a_chanceCreationCrossing',
       'a_chanceCreationShooting',
       'a_defencePressure',
       'a_defenceAggression',
       'a_defenceTeamWidth']] = pd.DataFrame([[np.nan, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan, np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]], index=matches.index)

h = 8
a = 17
t = 2
for row in range(len(matches)):
    #if matches.iat[row,1] in team_att['date'] and matches.iat[row,2] in team_att['team_api_id']:
    home = team_att.loc[((team_att['team_api_id'] == matches.iat[row,2]) & (team_att['date'] <= (matches.iat[row,1])))]
    away = team_att.loc[((team_att['team_api_id'] == matches.iat[row,3]) & (team_att['date'] <= (matches.iat[row,1])))]
    for i in range(9):
      if 2 > len(home) > 0:
        matches.iat[row, h+i] = home.iat[0,t+i]
      elif len(home) > 2:
        matches.iat[row, h+i] = home.iat[len(home)-1,t+i]
      if 2 > len(away) > 0:
        matches.iat[row, a+i] = away.iat[0,t+i]
      elif len(away) > 2:
        matches.iat[row, a+i] = away.iat[len(away)-1,t+i]

    
print(matches)

          season                 date  home_team_api_id  away_team_api_id  \
0      2008/2009  2008-08-17 00:00:00              9987              9993   
1      2008/2009  2008-08-16 00:00:00             10000              9994   
2      2008/2009  2008-08-16 00:00:00              9984              8635   
3      2008/2009  2008-08-17 00:00:00              9991              9998   
4      2008/2009  2008-08-16 00:00:00              7947              9985   
...          ...                  ...               ...               ...   
25974  2015/2016  2015-09-22 00:00:00             10190             10191   
25975  2015/2016  2015-09-23 00:00:00              9824             10199   
25976  2015/2016  2015-09-23 00:00:00              9956             10179   
25977  2015/2016  2015-09-22 00:00:00              7896             10243   
25978  2015/2016  2015-09-23 00:00:00             10192              9931   

       home_team_goal  away_team_goal   home_avg   away_avg  \
0           

fill na values

In [ ]:
numeric = ['h_buildUpPlaySpeed',
        'h_buildUpPlayPassing',
       'h_chanceCreationPassing',
       'h_chanceCreationCrossing',
       'h_chanceCreationShooting',
       'h_defencePressure',
       'h_defenceAggression',
       'h_defenceTeamWidth','a_buildUpPlaySpeed',
       'a_buildUpPlayPassing',
       'a_chanceCreationPassing',
       'a_chanceCreationCrossing',
       'a_chanceCreationShooting',
       'a_defencePressure',
       'a_defenceAggression',
       'a_defenceTeamWidth']

catagoric = [
       'h_buildUpPlayDribblingClass',
       'a_buildUpPlayDribblingClass']

matches[numeric] = matches[numeric].fillna(matches[numeric].mean())
matches[catagoric] = matches[catagoric].fillna(matches[catagoric].mode().iloc[0])

print(matches)

          season                 date  home_team_api_id  away_team_api_id  \
0      2008/2009  2008-08-17 00:00:00              9987              9993   
1      2008/2009  2008-08-16 00:00:00             10000              9994   
2      2008/2009  2008-08-16 00:00:00              9984              8635   
3      2008/2009  2008-08-17 00:00:00              9991              9998   
4      2008/2009  2008-08-16 00:00:00              7947              9985   
...          ...                  ...               ...               ...   
25974  2015/2016  2015-09-22 00:00:00             10190             10191   
25975  2015/2016  2015-09-23 00:00:00              9824             10199   
25976  2015/2016  2015-09-23 00:00:00              9956             10179   
25977  2015/2016  2015-09-22 00:00:00              7896             10243   
25978  2015/2016  2015-09-23 00:00:00             10192              9931   

       home_team_goal  away_team_goal   home_avg   away_avg  \
0           

factorize catagorical values

In [ ]:
matches['h_buildUpPlayDribblingClass'] = pd.factorize(matches['h_buildUpPlayDribblingClass'])[0]+1
matches['a_buildUpPlayDribblingClass'] = pd.factorize(matches['a_buildUpPlayDribblingClass'])[0]+1


          season  home_team_api_id  away_team_api_id   home_avg   away_avg  \
0      2008/2009              9987              9993  68.600000  68.600000   
1      2008/2009             10000              9994  68.600000  68.600000   
2      2008/2009              9984              8635  68.600000  68.600000   
3      2008/2009              9991              9998  68.600000  68.600000   
4      2008/2009              7947              9985  68.600000  68.600000   
...          ...               ...               ...        ...        ...   
25974  2015/2016             10190             10191  65.672727  67.727273   
25975  2015/2016              9824             10199  66.981818  69.880000   
25976  2015/2016              9956             10179  68.490909  69.727273   
25977  2015/2016              7896             10243  67.454545  70.418182   
25978  2015/2016             10192              9931  67.454545  66.436364   

       h_buildUpPlaySpeed  h_buildUpPlayDribblingClass  h_build

Add match outcome 1=home win, 0=draw, -1=away win

In [ ]:
matches['outcome'] = 0
matches.loc[matches['home_team_goal'] > matches['away_team_goal'], 'outcome'] = 1
matches.loc[matches['home_team_goal'] < matches['away_team_goal'], 'outcome'] = -1
print(matches)

          season                 date  home_team_api_id  away_team_api_id  \
0      2008/2009  2008-08-17 00:00:00              9987              9993   
1      2008/2009  2008-08-16 00:00:00             10000              9994   
2      2008/2009  2008-08-16 00:00:00              9984              8635   
3      2008/2009  2008-08-17 00:00:00              9991              9998   
4      2008/2009  2008-08-16 00:00:00              7947              9985   
...          ...                  ...               ...               ...   
25974  2015/2016  2015-09-22 00:00:00             10190             10191   
25975  2015/2016  2015-09-23 00:00:00              9824             10199   
25976  2015/2016  2015-09-23 00:00:00              9956             10179   
25977  2015/2016  2015-09-22 00:00:00              7896             10243   
25978  2015/2016  2015-09-23 00:00:00             10192              9931   

       home_team_goal  away_team_goal   home_avg   away_avg  \
0           

remove redundent columns: dates, goals, id

In [ ]:
matches = matches.drop('date', axis=1)
matches = matches.drop('home_team_goal', axis=1)
matches = matches.drop('away_team_goal', axis=1)
matches = matches.drop('home_team_api_id', axis=1)
matches = matches.drop('away_team_api_id', axis=1)

print(matches)

          season  home_team_api_id  away_team_api_id   home_avg   away_avg  \
0      2008/2009              9987              9993  68.600000  68.600000   
1      2008/2009             10000              9994  68.600000  68.600000   
2      2008/2009              9984              8635  68.600000  68.600000   
3      2008/2009              9991              9998  68.600000  68.600000   
4      2008/2009              7947              9985  68.600000  68.600000   
...          ...               ...               ...        ...        ...   
25974  2015/2016             10190             10191  65.672727  67.727273   
25975  2015/2016              9824             10199  66.981818  69.880000   
25976  2015/2016              9956             10179  68.490909  69.727273   
25977  2015/2016              7896             10243  67.454545  70.418182   
25978  2015/2016             10192              9931  67.454545  66.436364   

       h_buildUpPlaySpeed  h_buildUpPlayDribblingClass  h_build

split to train and test and do feature selection

In [ ]:
train = matches.loc[matches['season'] != '2015/2016']
test = matches.loc[matches['season'] == '2015/2016']
train = train.drop('season', axis=1)
test = test.drop('season', axis =1)


X_train = train.drop('outcome', axis=1)
y_train = train['outcome']
X_test = test.drop('outcome', axis=1)
y_test = test['outcome']
linreg = LinearRegression()
rfe = RFE(estimator=linreg, n_features_to_select=6)
rfe.fit(X_train, y_train)
print(X_train.columns[rfe.support_])

Index(['h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_defencePressure', 'a_buildUpPlayDribblingClass',
       'a_buildUpPlayPassing', 'a_defencePressure'],
      dtype='object')


#Test Different Models

##on all features

In [ ]:
models = [KNeighborsClassifier(n_neighbors=5), DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier()]
class_labels = ['Home', 'Draw', 'Away']

for model in models:
    # Compute the cross-validation scores
    scores = cross_val_score(model, X_train, y_train, cv=5)
    # Print the average cross-validation score
    print(f"Average cross-validation score for {type(model).__name__}: {scores.mean():.3f}")
    # Fit the model to the training data
    model.fit(X_train, y_train)
    # Use the model to make predictions on the test data
    y_pred = model.predict(X_test)
    # Compute the accuracy and f-measure scores
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    # Print the accuracy and f-measure scores
    print(f"Accuracy score for {type(model).__name__}: {acc:.3f}")
    print(f"F-measure score for {type(model).__name__}: {f1:.3f}")
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(f"Confusion matrix for {type(model).__name__}:")
    print(conf_df)


Average cross-validation score for KNeighborsClassifier: 0.369
Accuracy score for KNeighborsClassifier: 0.418
F-measure score for KNeighborsClassifier: 0.392
Confusion matrix for KNeighborsClassifier:
      Home  Draw  Away
Home   475   189   348
Draw   334   189   332
Away   443   289   727
Average cross-validation score for DecisionTreeClassifier: 0.357
Accuracy score for DecisionTreeClassifier: 0.361
F-measure score for DecisionTreeClassifier: 0.345
Confusion matrix for DecisionTreeClassifier:
      Home  Draw  Away
Home   331   272   409
Draw   255   226   374
Away   394   420   645
Average cross-validation score for RandomForestClassifier: 0.425
Accuracy score for RandomForestClassifier: 0.454
F-measure score for RandomForestClassifier: 0.370
Confusion matrix for RandomForestClassifier:
      Home  Draw  Away
Home   336    81   595
Draw   181    84   590
Away   236   133  1090
Average cross-validation score for GradientBoostingClassifier: 0.447
Accuracy score for GradientBoostingC

## with top 10 features

In [ ]:
train_10 = train[['home_avg', 'h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_chanceCreationPassing', 'h_chanceCreationShooting',
       'h_defencePressure', 'a_buildUpPlayDribblingClass',
       'a_buildUpPlayPassing', 'a_chanceCreationPassing', 'a_defencePressure']]
test_10 = train[['home_avg', 'h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_chanceCreationPassing', 'h_chanceCreationShooting',
       'h_defencePressure', 'a_buildUpPlayDribblingClass',
       'a_buildUpPlayPassing', 'a_chanceCreationPassing', 'a_defencePressure']]

X_train = train.drop('outcome', axis=1)
y_train = train['outcome']
X_test = test.drop('outcome', axis=1)
y_test = test['outcome']


for model in models:
    # Compute the cross-validation scores
    scores = cross_val_score(model, X_train, y_train, cv=5)
    # Print the average cross-validation score
    print(f"Average cross-validation score for {type(model).__name__}: {scores.mean():.3f}")
    # Fit the model to the training data
    model.fit(X_train, y_train)
    # Use the model to make predictions on the test data
    y_pred = model.predict(X_test)
    # Compute the accuracy and f-measure scores
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    # Print the accuracy and f-measure scores
    print(f"Accuracy score for {type(model).__name__}: {acc:.3f}")
    print(f"F-measure score for {type(model).__name__}: {f1:.3f}")
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(f"Confusion matrix for {type(model).__name__}:")
    print(conf_df)

Average cross-validation score for KNeighborsClassifier: 0.369
Accuracy score for KNeighborsClassifier: 0.418
F-measure score for KNeighborsClassifier: 0.392
Confusion matrix for KNeighborsClassifier:
      Home  Draw  Away
Home   475   189   348
Draw   334   189   332
Away   443   289   727
Average cross-validation score for DecisionTreeClassifier: 0.357
Accuracy score for DecisionTreeClassifier: 0.361
F-measure score for DecisionTreeClassifier: 0.346
Confusion matrix for DecisionTreeClassifier:
      Home  Draw  Away
Home   318   276   418
Draw   244   243   368
Away   391   428   640
Average cross-validation score for RandomForestClassifier: 0.419
Accuracy score for RandomForestClassifier: 0.446
F-measure score for RandomForestClassifier: 0.358
Confusion matrix for RandomForestClassifier:
      Home  Draw  Away
Home   317    79   616
Draw   214    77   564
Away   243   126  1090
Average cross-validation score for GradientBoostingClassifier: 0.447
Accuracy score for GradientBoostingC

##With top 6 features

In [ ]:
train_6 = train[['h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_defencePressure', 'a_buildUpPlayDribblingClass',
       'a_buildUpPlayPassing', 'a_defencePressure']]
test_6 = test[['h_buildUpPlayDribblingClass', 'h_buildUpPlayPassing',
       'h_defencePressure', 'a_buildUpPlayDribblingClass',
       'a_buildUpPlayPassing', 'a_defencePressure']]

X_train = train.drop('outcome', axis=1)
y_train = train['outcome']
X_test = test.drop('outcome', axis=1)
y_test = test['outcome']


for model in models:
    # Compute the cross-validation scores
    scores = cross_val_score(model, X_train, y_train, cv=5)
    # Print the average cross-validation score
    print(f"Average cross-validation score for {type(model).__name__}: {scores.mean():.3f}")
    # Fit the model to the training data
    model.fit(X_train, y_train)
    # Use the model to make predictions on the test data
    y_pred = model.predict(X_test)
    # Compute the accuracy and f-measure scores
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    # Print the accuracy and f-measure scores
    print(f"Accuracy score for {type(model).__name__}: {acc:.3f}")
    print(f"F-measure score for {type(model).__name__}: {f1:.3f}")
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(f"Confusion matrix for {type(model).__name__}:")
    print(conf_df)

Average cross-validation score for KNeighborsClassifier: 0.369
Accuracy score for KNeighborsClassifier: 0.418
F-measure score for KNeighborsClassifier: 0.392
Confusion matrix for KNeighborsClassifier:
      Home  Draw  Away
Home   475   189   348
Draw   334   189   332
Away   443   289   727
Average cross-validation score for DecisionTreeClassifier: 0.354
Accuracy score for DecisionTreeClassifier: 0.359
F-measure score for DecisionTreeClassifier: 0.339
Confusion matrix for DecisionTreeClassifier:
      Home  Draw  Away
Home   312   287   413
Draw   259   216   380
Away   388   405   666
Average cross-validation score for RandomForestClassifier: 0.422
Accuracy score for RandomForestClassifier: 0.457
F-measure score for RandomForestClassifier: 0.370
Confusion matrix for RandomForestClassifier:
      Home  Draw  Away
Home   335    84   593
Draw   197    79   579
Away   209   144  1106
Average cross-validation score for GradientBoostingClassifier: 0.447
Accuracy score for GradientBoostingC